# Features based on literature

In [2]:
import pandas as pd
%run "..\Model\DataHelpers.ipynb"

print(f'Loading gene data - Start')
df = pd.read_csv('../Data/geneDataPreProcessed.csv')
print(f'Loading gene data - End')

Loading gene data - Start
Loading gene data - End


In [3]:
variant = FeatureVariant.LITERATURE

features = [ 'TBC1D9', 'GATA3', 'SLC16A6', 'ESR1', 'INPP4B', 
             'SLC44A4', 'ANXA9', 'AGR2', 'MCCC2', 'TSPAN1',
             'STBD1', 'MLPH', 'CACNA2D2', 'RARA', 'STARD3', 
             'PPP1R14C', 'LDHB', 'MFGE8', 'PSAT1' ]

# Send dataframe to a csv

In [5]:
df[features+constTargetAndMetadata].to_csv(f'../Data/patient_genes_{variant}.csv', index=False)